In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# intents and responses
intents = {
    "greeting": ["হ্যালো", "hello", "hi"],
    "help": ["সাহায্য করতে পারবেন?", "sahajjo korte parben?"],
    "work": ["generate  code",  " code likhen"],
    "thanks": ["ধন্যবাদ", "Thanks"],
    
}

responses = {
    "greeting": ["হ্যালো","hello", "hi"],
     "help": ["হ্যা,আমি আপনাকে কিভাবে সাহায্য করতে পারি", "Hae, Apnake kivabe sahajjo korte pari?"],
    "work": ["print(\"Hello, Python DEV!\")","print(\"Hello, Python DEV!\")"],
    "thanks": ["স্বাগতম", "Wellcome"],
   
}


lemmatizer = WordNetLemmatizer()


stop_words = set(stopwords.words('english'))


def tokenize(sentence):
    tokens = word_tokenize(sentence)
    tokens = [token.lower() for token in tokens] # Convert to lowercase for consistency
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens


vectorizer = TfidfVectorizer()

# Prepare the training data
train_x = []
train_y = []

for intent, sentences in intents.items():
    tokenized_sentences = [" ".join(tokenize(sentence)) for sentence in sentences]
    train_x.extend(tokenized_sentences)
    train_y.extend([list(intents.keys()).index(intent)] * len(sentences))

train_x = vectorizer.fit_transform(train_x).toarray()
train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(intents))


input_shape = train_x.shape[1]


model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,)),
    Dense(64, activation='relu'),
    Dense(len(intents), activation='softmax')
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)


model.save("chatbotmodel.h5")

# chatbot function
def chatbot(input_text):
   
    tokens = tokenize(input_text)
    
    
    known_words = set(vectorizer.vocabulary_.keys())
    if not any(token in known_words for token in tokens):
        return "Your input response is not included in my training data."

   
    numerical_input = vectorizer.transform([" ".join(tokens)]).toarray()

    # prediction using the model
    prediction = model.predict(numerical_input)
    max_probability = np.max(prediction)
    threshold = 0.6  

    if max_probability < threshold:
        response = "Your input response is not included in my training data."
    else:
       
        index = np.argmax(prediction)

    
        intent = list(intents.keys())[index]
        
       
        if any(word in input_text for word in intents[intent]):
          response = random.choice([r for r in responses[intent] if r in intents[intent]])
        else:
          response = random.choice([r for r in responses[intent] if r not in intents[intent]])
    return response

# Test the chatbot
while True:
    input_text = input("You: ")
    response = chatbot(input_text)
    print("You:", input_text)
    print("Chatbot:", response)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/200


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4963 - loss: 1.3675
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2889 - loss: 1.3485 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8519 - loss: 1.3085 
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7852 - loss: 1.2598 
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8593 - loss: 1.2442 
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8593 - loss: 1.2307 
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9259 - loss: 1.1963 
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.1626 
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.1433 
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.1047 
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.0897 
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.0697 
E

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
You: hi
Chatbot: হ্যালো
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
You: hi
Chatbot: হ্যালো
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
You: hello
Chatbot: হ্যালো
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
You: hi
Chatbot: হ্যালো
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
You: sahajjo korte parben
Chatbot: Hae, Apnake kivabe sahajjo korte pari?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
You: code likhen
Chatbot: print("Hello, Python DEV!")
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
You: code likhen
Chatbot: print("Hello, Python DEV!")
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
You: thanks
Chatbot: Wellcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
You: thanks
Chatbot: Wellcome
